## Individual-based model (Polechova and Barton, 2015)

In [1]:
using Random, Distributions, Plots

In [49]:
#global variables cfr. table 1

rm = 1.025 #maximum intrinsic growth rate
Vs = 2 #variance of stabilising selection 

b = 0.1 #gradient in the environmental optimum
σ = sqrt(0.5) #dispersal per generation
K = 300 #carrying capacity for a well-adapted phenotype
#α = 1/sqrt(20) #allelic effect
α = 1
μ = 10^-6 #mutation rate
T = 5000 #number of generations 
h_ = 1 # ??

1

In [50]:
num_demes(σ,Vs,α) = round(Int64,10*4*sqrt(σ^2*Vs)/α)
num_loci(nd,b,α) = round(Int64,nd*b/α)
num_ind(K,b,σ,h,Vs,rm) = round(Int64,K*((1-b*σ)/(2*h^2*sqrt(Vs)*rm)))

num_ind (generic function with 1 method)

In [51]:
nd = num_demes(σ,Vs,α)

40

In [52]:
nl = num_loci(nd,b,α)

4

In [53]:
nn = num_ind(K,b,σ,h_,Vs,rm)

96

In [54]:
mutable struct Agent{T}
    loci::Vector{T}
end

Note: allele effect size ? -> L bi-allelic loci underlying trait x, allele effect size 0 or α -> trait = sum of allele effect sizes over all loci


In [55]:
pushlocus!(a::Agent, x) = push!(a.loci, x)

pushlocus! (generic function with 1 method)

In [56]:
#generate N haploid individuals with L loci and two possible allele-effect sizes α or 0
function make_agents(N,L,α)
    pop = [Agent(α .* rand(Bool, L)) for i in 1:N]
    return pop
end

make_agents (generic function with 1 method)

In [57]:
agents = make_agents(10,10,1)

10-element Array{Agent{Int64},1}:
 Agent{Int64}([0, 1, 0, 1, 0, 1, 1, 1, 0, 0])
 Agent{Int64}([0, 1, 1, 0, 0, 0, 1, 1, 1, 0])
 Agent{Int64}([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
 Agent{Int64}([0, 0, 0, 0, 0, 0, 0, 0, 1, 1])
 Agent{Int64}([1, 0, 0, 1, 0, 1, 0, 1, 0, 0])
 Agent{Int64}([0, 0, 0, 0, 0, 0, 1, 0, 0, 1])
 Agent{Int64}([1, 0, 0, 1, 1, 0, 1, 0, 1, 1])
 Agent{Int64}([1, 1, 0, 1, 0, 1, 1, 0, 0, 0])
 Agent{Int64}([1, 0, 1, 0, 1, 1, 0, 0, 0, 0])
 Agent{Int64}([1, 1, 0, 1, 1, 0, 1, 1, 0, 1])

In [58]:
struct Deme{A,T}
    agents::Vector{A}
    carrying_capacity::Int64
    optimal_phenotype::T 
end

In [59]:
d = Deme(agents, 100, 0.0)

Deme{Agent{Int64},Float64}(N=10)

In [60]:
Base.length(d::Deme) = length(d.agents)
Base.show(io::IO, d::Deme{A,T}) where {A,T} = write(io, "Deme{$A,$T}(N=$(length(d)))")
Base.push!(d::Deme, a::Agent) = push!(d.agents, a)

In [61]:
push!(d, Agent(1 .* rand(Bool, 10)))
d

Deme{Agent{Int64},Float64}(N=11)

In [62]:
struct Habitat{T}
    demes::Vector{T}
end

In [63]:
#generate a single deme with population of size N, L loci, α allelic effect, carrying capacity K and optimal phenotype θ
function make_deme(N,L,α,K,θ)
    pop = make_agents(N,L,α)
    return Deme(pop,K,θ)
end
    

make_deme (generic function with 1 method)

In [64]:
w = make_deme(5,2,1,150,10)

Deme{Agent{Int64},Int64}(N=5)

In [65]:
function pop_size(d::Deme)
    return length(d.agents)
end

pop_size (generic function with 1 method)

In [66]:
pop_size(w)

5

In [67]:
#generate a linear 1D habitat of x demes with equal pop size (in paper the pop starts in center of habitat)
function make_habitat(x,N,L,α,K,θ)
    hab = [make_deme(N,L,α,K,θ) for i in 1:x]
    return Habitat(hab)
end

make_habitat (generic function with 1 method)

In [68]:
h = make_habitat(5,5,2,1,150,10)

Habitat (n=5):
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)

In [69]:
Base.length(h::Habitat) = length(h.demes)
Base.show(io::IO, h::Habitat) = write(io, 
    "Habitat (n=$(length(h))):\n  $(join(string.(h.demes), "\n  "))")

In [70]:
h

Habitat (n=5):
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)

In [71]:
#habitat with 7 demes and starting pop in central deme
h = make_habitat(7,0,0,0,150,10)
p = make_agents(3,3,1)
push!(h.demes[4].agents, agents ...)
h

Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=11)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)

## Migration

In [72]:
#should be diffusive migration with a Gaussian dispersal kernel (truncated at 2 SD)
emptycopy(d::Deme{A}) where A = Deme(A[], d.carrying_capacity, d.optimal_phenotype)
emptycopy(h::Habitat) = Habitat(emptycopy.(h.demes))

function migrate(h::Habitat, p)
    new_h = emptycopy(h)
    for (i, deme) in enumerate(h.demes)
        for agent in deme.agents
            step = rand() < p ? rand([-1,1]) : 0 
            if step == -1 && i == 1
                step = 0
            elseif step == 1  && i == length(h)
                step = 0
            end
            push!(new_h.demes[i+step], agent)
        end
    end
    new_h
end

migrate (generic function with 1 method)

In [73]:
for i=1:5
    h = migrate(h, 0.2)
    @show h
end

h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=1)
  Deme{Agent{Int64},Int64}(N=8)
  Deme{Agent{Int64},Int64}(N=2)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=2)
  Deme{Agent{Int64},Int64}(N=6)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=1)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=2)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Ag

## Selection

In [83]:
a = Agent(1 .* rand(Bool, 10))

Agent{Int64}([1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [84]:
phenotype(a::Agent) = sum(a.loci)
phenotype(a)

9

In [85]:
Base.sum(a::Agent) = sum(a.loci)
sum(a)

9

In [86]:
function malthusian_fitness(d::Deme,a::Agent,rm,Vs)
    N = length(d)
    K = d.carrying_capacity
    θ = d.optimal_phenotype
    z = sum(a)
    return rm*(1-(N/K))-((z-θ)^2)/(2*Vs)
end   

malthusian_fitness (generic function with 1 method)

In [87]:
d = make_deme(5,10,α,K,1)

Deme{Agent{Int64},Int64}(N=5)

In [126]:
number_of_offspring(d::Deme,a::Agent,rm,Vs) = rand(Poisson(exp(malthusian_fitness(d::Deme,a::Agent,rm,Vs))))

number_of_offspring (generic function with 1 method)

In [132]:
for agent in d.agents
    @show number_of_offspring(d,agent,rm,Vs)
end

number_of_offspring(d, agent, rm, Vs) = 0
number_of_offspring(d, agent, rm, Vs) = 4
number_of_offspring(d, agent, rm, Vs) = 3
number_of_offspring(d, agent, rm, Vs) = 0
number_of_offspring(d, agent, rm, Vs) = 1


## Mutation

In [133]:
function mutate(a::Agent,μ)
   #each allele in a.loci has probability of mu to flip 
    empty_agent = Agent(Int64[])
    for x in a.loci
        if rand() > μ
            pushlocus!(empty_agent,x)
        else
            x == 0 ? x = α : x = 0
            pushlocus!(empty_agent,x)
        end
    end
    empty_agent
end

mutate (generic function with 1 method)

In [134]:
for i=1:5
    a = mutate(a, 0.5)
    @show a
end

a = Agent{Int64}([0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
a = Agent{Int64}([0, 1, 0, 1, 0, 1, 1, 0, 0, 0])
a = Agent{Int64}([1, 0, 1, 0, 1, 1, 1, 0, 1, 1])
a = Agent{Int64}([0, 0, 1, 0, 0, 0, 0, 1, 0, 1])
a = Agent{Int64}([0, 0, 0, 1, 1, 1, 0, 0, 0, 1])


## Recombination

In [135]:
#unlinked loci, r = 0.5
function recombine(a::Agent,b::Agent)
    empty_agent = Agent(Int64[])
    for i in 1:length(a.loci)
        rand() > 0.5 ? pushlocus!(empty_agent,a.loci[i]) : pushlocus!(empty_agent,b.loci[i])
    end
    empty_agent
end
    

recombine (generic function with 1 method)

In [136]:
aa = Agent([1 for i in 1:10])
bb = Agent([0 for i in 1:10])

Agent{Int64}([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [137]:
for i=1:5
    aa = recombine(aa, bb)
    @show aa
end

aa = Agent{Int64}([0, 0, 1, 1, 1, 0, 0, 1, 0, 1])
aa = Agent{Int64}([0, 0, 1, 0, 1, 0, 0, 0, 0, 1])
aa = Agent{Int64}([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
aa = Agent{Int64}([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
aa = Agent{Int64}([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
